In [ ]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import signal
import scipy.io.wavfile as wavfile
from scipy.signal import resample, correlate, hilbert
from PIL import Image


from util.plotting import compute_fft_plot_from_sample_rate
from util.data_io import read_rtl_raw_data, read_gqrx_raw_data
from util.filtering import low_pass_filter_complex_signal, low_pass_filter_real_signal
from util.demodulation import chunked_demodulate_signal
from util.phase_locked_loop import phase_locked_loop

In [ ]:
class LowPassFilter():
    def __init__(self, frequency_cutoff: float, sample_rate: int, order: int) -> None:
        self.b,self.a = signal.butter(N=order, Wn=frequency_cutoff, fs=sample_rate)
        self.y = np.zeros((len(self.a) - 1,))
        self.x = np.zeros((len(self.b),))
    
    def step(self, x: float):
        self.x = np.concatenate([[x], self.x[:-1]])
        y = (1/self.a[0]) * (self.x.dot(self.b) - self.y.dot(self.a[1:]))
        self.y = np.concatenate([[y], self.y[:-1]])

        return y

In [ ]:
sample_rate = 250

time = np.arange(sample_rate * 5) / sample_rate
test_frequency = 10.0
test_signal = np.sin(2 * np.pi * test_frequency * time)


In [ ]:
fig = go.Figure()
fig.add_scattergl(x=time,y=test_signal)
fig.show()

In [ ]:
phase = 0
frequency = 9.9 * (2*np.pi / sample_rate)
gain = 0.00001

output = []
errors = []
frequencies = []

lpf_i = LowPassFilter(frequency_cutoff=3, sample_rate=sample_rate, order=5)
lpf_q = LowPassFilter(frequency_cutoff=3, sample_rate=sample_rate, order=5)

i = 0

for x in test_signal:
    phase_sin = np.sin(phase)
    phase_cos = np.cos(phase)

    output.append(phase_sin)

    error_i = lpf_i.step(phase_sin * x)
    error_q = lpf_q.step(phase_cos * x)
    error = np.arctan2(error_q, error_i)

    errors.append(error)
    # error = 0

    frequency += gain * error
    frequencies.append(frequency)

    phase = np.mod(phase + frequency, 2*np.pi)

    # print(f"psin: {phase_sin:.2f} pcos: {phase_cos:.2f}")
    # print(f"error_i: {error_i:.6f} error_q: {error_q:.6f}")
    # print(f"frequency_update:{gain * error:.6f}")
    # print("")

    # if i > 5:
    #     break
    # i+=1


In [ ]:


fig = go.Figure()
fig.add_scattergl(x=time,y=test_signal, name="ref")
fig.add_scattergl(x=time,y=output, name="out")
fig.add_scattergl(x=time,y=frequencies, name="freq")
fig.add_scattergl(x=time,y=errors, name="error")



fig.show()

In [ ]:
phase = 0
frequency = 9.9 * (2*np.pi / sample_rate)
gain = 0.00001

output = []
errors = []
frequencies = []

lpf_i = LowPassFilter(frequency_cutoff=3, sample_rate=sample_rate, order=5)
lpf_q = LowPassFilter(frequency_cutoff=3, sample_rate=sample_rate, order=5)

i = 0

for x in test_signal:
    phase_sin = np.sin(phase)
    phase_cos = np.cos(phase)

    output.append(phase_sin)

    error_i = lpf_i.step(phase_sin * x)
    error_q = lpf_q.step(phase_cos * x)
    error = np.arctan2(error_q, error_i)

    errors.append(error)
    # error = 0

    frequency += gain * error
    frequencies.append(frequency)

    phase = np.mod(phase + frequency, 2*np.pi)

    # print(f"psin: {phase_sin:.2f} pcos: {phase_cos:.2f}")
    # print(f"error_i: {error_i:.6f} error_q: {error_q:.6f}")
    # print(f"frequency_update:{gain * error:.6f}")
    # print("")

    # if i > 5:
    #     break
    # i+=1
